In [9]:
import foolbox as fb
import art
import tensorflow as tf
import numpy as np
import joblib

In [10]:
X_train = joblib.load('../../preprocessed_data/InSDN/X_train.joblib')
X_test = joblib.load('../../preprocessed_data/InSDN/X_test.joblib')
Y_train = joblib.load('../../preprocessed_data/InSDN/Y_train.joblib')
Y_test = joblib.load('../../preprocessed_data/InSDN/Y_test.joblib')

print (X_test[:10])
print (Y_test[:10])

[[[-9.27842021e-01]
  [-4.94954705e-01]
  [-3.08426678e-01]
  [-3.91555345e-03]
  [-3.69372331e-02]
  [-9.75534786e-03]
  [-2.31714882e-02]
  [-1.74138755e-01]
  [-1.49991870e-01]
  [-1.64555863e-01]
  [-1.54659420e-01]
  [-1.41909316e-01]
  [-2.45230436e-01]
  [-2.43889943e-01]
  [-2.25802675e-01]
  [-1.08766146e-01]
  [-2.40492210e-01]
  [-2.69861877e-01]
  [-2.91075468e-01]
  [-3.00794810e-01]
  [-2.06512958e-02]
  [-1.78727910e-01]
  [-1.39054716e-01]
  [-1.45615727e-01]
  [-1.64919034e-01]
  [-3.38274911e-02]
  [-3.01781595e-01]
  [-2.69620836e-01]
  [-2.84314632e-01]
  [-2.96600550e-01]
  [-1.94003452e-02]
  [ 0.00000000e+00]
  [-2.38540351e-01]
  [ 0.00000000e+00]
  [-1.07913181e-01]
  [-5.85533530e-02]
  [-5.40889427e-02]
  [-6.39446676e-02]
  [-2.37514764e-01]
  [-2.88715899e-01]
  [-1.56737357e-01]
  [-2.78276533e-01]
  [-2.40975961e-01]
  [-4.46181968e-02]
  [-3.98650169e-01]
  [-5.57213366e-01]
  [-3.26919332e-02]
  [-2.38540351e-01]
  [-6.14069104e-01]
  [-1.07913181e-01]


In [11]:
from tensorflow.keras.models import load_model

cnn_loaded = load_model('../../models/cnn_insdn_model.keras')
lstm_loaded = load_model('../../models/lstm_insdn_model.keras')
ae_lstm_loaded = load_model('../../models/ae-lstm_insdn_model.keras')

In [ ]:
from art.attacks.evasion import FeatureAdversariesTensorFlowV2, HopSkipJump, BoundaryAttack
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_mnist
from tensorflow.keras.utils import to_categorical

classifier = TensorFlowV2Classifier(
    model=ae_lstm_loaded,
    nb_classes=2,
    input_shape=(X_test.shape[1], X_test.shape[2]),
    loss_object=tf.keras.losses.BinaryCrossentropy(),
    clip_values=(0.0, 1.0)
)


# si Y_test est de forme (n,) avec 0 ou 1 :
Y_test_categorical = to_categorical(Y_test, num_classes=2)


attack_hsj = HopSkipJump(classifier=classifier)
adv_hopskipjump = attack_hsj.generate(x=X_test[:10])
attack_boundary = BoundaryAttack(estimator=classifier)
adv_boundary = attack_boundary.generate(x=X_test[:10], y=Y_test[:10])

# original predictions
preds_orig = classifier.predict(X_test[:10])

# HopSkipJump predictions
preds_adv_hsj = classifier.predict(adv_hopskipjump)

# BoundaryAttack predictions
preds_adv_boundary = classifier.predict(adv_boundary)

print("Original predictions       :", np.argmax(preds_orig, axis=1))
print("After HopSkipJump attack   :", np.argmax(preds_adv_hsj, axis=1))
print("After BoundaryAttack       :", np.argmax(preds_adv_boundary, axis=1))

HopSkipJump:   0%|          | 0/20 [00:17<?, ?it/s]


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/bossantm/anaconda3/envs/SDNIOT-env/lib/python3.10/site-packages/keras/src/backend.py", line 5160, in <genexpr>
    output_ta_t = tuple(  File "/home/bossantm/anaconda3/envs/SDNIOT-env/lib/python3.10/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


KeyboardInterrupt: 